In [9]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,roc_curve
from sklearn import preprocessing
import warnings

warnings.filterwarnings("ignore")

import time


def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt


def base_process(data):
    lbl = preprocessing.LabelEncoder()
    print(
        '--------------------------------------------------------------item--------------------------------------------------------------')
    data['len_item_category'] = data['item_category_list'].map(lambda x: len(str(x).split(';')))
    data['len_item_property'] = data['item_property_list'].map(lambda x: len(str(x).split(';')))
    for i in range(1, 3):
        data['item_category_list' + str(i)] = lbl.fit_transform(data['item_category_list'].map(
            lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))  # item_category_list的第0列全部都一样
    for i in range(10):
        data['item_property_list' + str(i)] = lbl.fit_transform(data['item_property_list'].map(lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))
    for col in ['item_id', 'item_brand_id', 'item_city_id']:
        data[col] = lbl.fit_transform(data[col])
    print(
        '--------------------------------------------------------------user--------------------------------------------------------------')
    for col in ['user_id']:
        data[col] = lbl.fit_transform(data[col])
    print('user 0,1 feature')
    data['gender0'] = data['user_gender_id'].apply(lambda x: 1 if x == -1 else 2)
    data['age0'] = data['user_age_level'].apply(lambda x: 1 if x == 1004 | x == 1005 | x == 1006 | x == 1007  else 2)
    data['occupation0'] = data['user_occupation_id'].apply(lambda x: 1 if x == -1 | x == 2003  else 2)
    data['star0'] = data['user_star_level'].apply(lambda x: 1 if x == -1 | x == 3000 | x == 3001  else 2)
    print(
        '--------------------------------------------------------------context--------------------------------------------------------------')
    data['realtime'] = data['context_timestamp'].apply(timestamp_datetime)
    data['realtime'] = pd.to_datetime(data['realtime'])
    data['day'] = data['realtime'].dt.day
    data['hour'] = data['realtime'].dt.hour
    data['len_predict_category_property'] = data['predict_category_property'].map(lambda x: len(str(x).split(';')))
    for i in range(5):
        data['predict_category_property' + str(i)] = lbl.fit_transform(data['predict_category_property'].map(
            lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))
    print('context 0,1 feature')
    data['context_page0'] = data['context_page_id'].apply(
        lambda x: 1 if x == 4001 | x == 4002 | x == 4003 | x == 4004 | x == 4007  else 2)
    print(
        '--------------------------------------------------------------shop--------------------------------------------------------------')
    for col in ['shop_id']:
        data[col] = lbl.fit_transform(data[col])
    data['shop_score_delivery0'] = data['shop_score_delivery'].apply(lambda x: 0 if x <= 0.98 and x >= 0.96  else 1)
    return data


def map_hour(x):
    if (x>=7)&(x<=12):
        return 1
    elif (x>=13)&(x<=20):
        return 2
    else:
        return 3

def deliver(x):
    #x=round(x,6)
    jiange=0.1
    for i in range(1,20):
        if (x>=4.1+jiange*(i-1))&(x<=4.1+jiange*i):
            return i+1
    if x==-5:
        return 1

def deliver1(x):
    if (x>=2)&(x<=4):
        return 1
    elif (x>=5)&(x<=7):
        return 2
    else:
        return 3


def review(x):
    # x=round(x,6)
    jiange = 0.02
    for i in range(1, 30):
        if (x >= 0.714 + jiange * (i - 1)) & (x <= 0.714 + jiange * i):
            return i + 1
    if x == -1:
        return 1

def review1(x):
    # x=round(x,6)
    if (x>=2)&(x<=12):
        return 1
    elif (x>=13)&(x<=15):
        return 2
    else:
        return 3


def service(x):
    #x=round(x,6)
    jiange=0.1
    for i in range(1,20):
        if (x>=3.93+jiange*(i-1))&(x<=3.93+jiange*i):
            return i+1
    if x==-1:
        return 1

def service1(x):
    if (x>=2)&(x<=7):
        return 1
    elif (x>=8)&(x<=9):
        return 2
    else:
        return 3


def describe(x):
    #x=round(x,6)
    jiange=0.1
    for i in range(1,30):
        if (x>=3.93+jiange*(i-1))&(x<=3.93+jiange*i):
            return i+1
    if x==-1:
        return 1

def describe1(x):
    if (x>=2)&(x<=8):
        return 1
    elif (x>=9)&(x<=10):
        return 2
    else:
        return 3

def shijian(data):
    data['hour_map'] = data['hour'].apply(map_hour)
    return data

def shop_fenduan(data):
    data['shop_score_delivery'] = data['shop_score_delivery'] * 5
    data = data[data['shop_score_delivery'] != -5]
    data['deliver_map'] = data['shop_score_delivery'].apply(deliver)
    data['deliver_map'] = data['deliver_map'].apply(deliver1)
    # del data['shop_score_delivery']
    print(data.deliver_map.value_counts())

    data['shop_score_service'] = data['shop_score_service'] * 5
    data = data[data['shop_score_service'] != -5]
    data['service_map'] = data['shop_score_service'].apply(service)
    data['service_map'] = data['service_map'].apply(service1)
    # del data['shop_score_service']
    print(data.service_map.value_counts())  # 视为好评，中评，差评
    #
    data['shop_score_description'] = data['shop_score_description'] * 5
    data = data[data['shop_score_description'] != -5]
    data['de_map'] = data['shop_score_description'].apply(describe)
    data['de_map'] = data['de_map'].apply(describe1)
    # del data['shop_score_description']
    print(data.de_map.value_counts())

    data = data[data['shop_review_positive_rate'] != -1]
    data['review_map'] = data['shop_review_positive_rate'].apply(review)
    data['review_map'] = data['review_map'].apply(review1)
    print(data.review_map.value_counts())

    data['normal_shop'] = data.apply(
        lambda x: 1 if (x.deliver_map == 3) & (x.service_map == 3) & (x.de_map == 3) & (x.review_map == 3) else 0,
        axis=1)
    del data['de_map']
    del data['service_map']
    del data['deliver_map']
    del data['review_map']
    return data


def slide_cnt(data):
    # item_cnt = data.groupby(by='item_id').count()['instance_id'].to_dict()
    # data['item_cnt'] = data['item_id'].apply(lambda x: item_cnt[x])
    # user_cnt = data.groupby(by='user_id').count()['instance_id'].to_dict()
    # data['user_cnt'] = data['user_id'].apply(lambda x: user_cnt[x])
    # shop_cnt = data.groupby(by='shop_id').count()['instance_id'].to_dict()
    # data['shop_cnt'] = data['shop_id'].apply(lambda x: shop_cnt[x])

    print('当前日期前一天的cnt')
    for d in range(19, 26):  # 18到24号
        df1 = data[data['day'] == d - 1]
        df2 = data[data['day'] == d]  # 19到25号
        user_cnt = df1.groupby(by='user_id').count()['instance_id'].to_dict()
        item_cnt = df1.groupby(by='item_id').count()['instance_id'].to_dict()
        shop_cnt = df1.groupby(by='shop_id').count()['instance_id'].to_dict()
        df2['user_cnt1'] = df2['user_id'].apply(lambda x: user_cnt.get(x, 0))
        df2['item_cnt1'] = df2['item_id'].apply(lambda x: item_cnt.get(x, 0))
        df2['shop_cnt1'] = df2['shop_id'].apply(lambda x: shop_cnt.get(x, 0))
        df2 = df2[['user_cnt1', 'item_cnt1', 'shop_cnt1', 'instance_id']]
        if d == 19:
            Df2 = df2
        else:
            Df2 = pd.concat([df2, Df2])
    data = pd.merge(data, Df2, on=['instance_id'], how='left')
    print('当前日期之前的cnt')
    for d in range(19, 26):
        # 19到25，25是test
        df1 = data[data['day'] < d]
        df2 = data[data['day'] == d]
        user_cnt = df1.groupby(by='user_id').count()['instance_id'].to_dict()
        item_cnt = df1.groupby(by='item_id').count()['instance_id'].to_dict()
        shop_cnt = df1.groupby(by='shop_id').count()['instance_id'].to_dict()
        df2['user_cntx'] = df2['user_id'].apply(lambda x: user_cnt.get(x, 0))
        df2['item_cntx'] = df2['item_id'].apply(lambda x: item_cnt.get(x, 0))
        df2['shop_cntx'] = df2['shop_id'].apply(lambda x: shop_cnt.get(x, 0))
        df2 = df2[['user_cntx', 'item_cntx', 'shop_cntx', 'instance_id']]
        if d == 19:
            Df2 = df2
        else:
            Df2 = pd.concat([df2, Df2])
    data = pd.merge(data, Df2, on=['instance_id'], how='left')

    print("前一个小时的统计量")

    return data


def zuhe(data):
    for col in ['user_gender_id','user_age_level','user_occupation_id','user_star_level']:
        data[col] = data[col].apply(lambda x: 0 if x == -1 else x)

    for col in ['item_sales_level', 'item_price_level', 'item_collected_level',
                'user_gender_id','user_age_level','user_occupation_id','user_star_level',
                'shop_review_num_level', 'shop_star_level']:
        data[col] = data[col].astype(str)

    print('item两两组合')
    data['sale_price'] = data['item_sales_level'] + data['item_price_level']
    data['sale_collect'] = data['item_sales_level'] + data['item_collected_level']
    data['price_collect'] = data['item_price_level'] + data['item_collected_level']

    print('user两两组合')
    data['gender_age'] = data['user_gender_id'] + data['user_age_level']
    data['gender_occ'] = data['user_gender_id'] + data['user_occupation_id']
    data['gender_star'] = data['user_gender_id'] + data['user_star_level']

    print('shop两两组合')
    data['review_star'] = data['shop_review_num_level'] + data['shop_star_level']


    for col in ['item_sales_level', 'item_price_level', 'item_collected_level',  'sale_price','sale_collect', 'price_collect',
                'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level','gender_age','gender_occ','gender_star',
                'shop_review_num_level','shop_star_level','review_star']:
        data[col] = data[col].astype(int)

    del data['review_star']

    return data

def item(data):
    print('一个item有多少brand,price salse collected level……')

    itemcnt = data.groupby(['item_id'], as_index=False)['instance_id'].agg({'item_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['item_id'], how='left')

    for col in ['item_brand_id','item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level']:
        itemcnt = data.groupby([col, 'item_id'], as_index=False)['instance_id'].agg({str(col) + '_item_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'item_id'], how='left')
        data[str(col) + '_item_prob']=data[str(col) + '_item_cnt']/data['item_cnt']
    del data['item_cnt']

    print('一个brand有多少price salse collected level……')

    itemcnt = data.groupby(['item_brand_id'], as_index=False)['instance_id'].agg({'item_brand_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['item_brand_id'], how='left')

    for col in ['item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level']:
        itemcnt = data.groupby([col, 'item_brand_id'], as_index=False)['instance_id'].agg({str(col) + '_brand_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'item_brand_id'], how='left')
        data[str(col) + '_brand_prob'] = data[str(col) + '_brand_cnt'] / data['item_brand_cnt']
    del data['item_brand_cnt']

    print('一个city有多少item_price_level，item_sales_level，item_collected_level，item_pv_level')

    itemcnt = data.groupby(['item_city_id'], as_index=False)['instance_id'].agg({'item_city_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['item_city_id'], how='left')
    for col in ['item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level']:
        itemcnt = data.groupby([col, 'item_city_id'], as_index=False)['instance_id'].agg({str(col) + '_city_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'item_city_id'], how='left')
        data[str(col) + '_city_prob'] = data[str(col) + '_city_cnt'] / data['item_city_cnt']
    del data['item_city_cnt']

    print('一个price有多少item_sales_level，item_collected_level，item_pv_level')

    itemcnt = data.groupby(['item_price_level'], as_index=False)['instance_id'].agg({'item_price_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['item_price_level'], how='left')
    for col in ['item_sales_level', 'item_collected_level', 'item_pv_level']:
        itemcnt = data.groupby([col, 'item_city_id'], as_index=False)['instance_id'].agg({str(col) + '_price_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'item_city_id'], how='left')
        data[str(col) + '_price_prob'] = data[str(col) + '_price_cnt'] / data['item_price_cnt']
    del data['item_price_cnt']

    print('一个item_sales_level有多少item_collected_level，item_pv_level')

    itemcnt = data.groupby(['item_sales_level'], as_index=False)['instance_id'].agg({'item_salse_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['item_sales_level'], how='left')
    for col in ['item_collected_level', 'item_pv_level']:
        itemcnt = data.groupby([col, 'item_sales_level'], as_index=False)['instance_id'].agg({str(col) + '_salse_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'item_sales_level'], how='left')
        data[str(col) + '_salse_prob'] = data[str(col) + '_salse_cnt'] / data['item_salse_cnt']
    del data['item_salse_cnt']

    print('一个item_collected_level有多少item_pv_level')

    itemcnt = data.groupby(['item_collected_level'], as_index=False)['instance_id'].agg({'item_coll_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['item_collected_level'], how='left')
    for col in ['item_pv_level']:
        itemcnt = data.groupby([col, 'item_collected_level'], as_index=False)['instance_id'].agg({str(col) + '_coll_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'item_collected_level'], how='left')
        data[str(col) + '_coll_prob'] = data[str(col) + '_coll_cnt'] / data['item_coll_cnt']
    del data['item_coll_cnt']

    return data

def user(data):
    print('用户有多少性别')
    itemcnt = data.groupby(['user_id'], as_index=False)['instance_id'].agg({'user_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_id'], how='left')

    for col in ['user_gender_id','user_age_level', 'user_occupation_id', 'user_star_level']:
        itemcnt = data.groupby([col, 'user_id'], as_index=False)['instance_id'].agg({str(col) + '_user_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'user_id'], how='left')
        data[str(col) + '_user_prob']=data[str(col) + '_user_cnt']/data['user_cnt']
    del data['user_cnt']

    print('性别的年龄段，职业有多少')
    itemcnt = data.groupby(['user_gender_id'], as_index=False)['instance_id'].agg({'user_gender_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_gender_id'], how='left')

    for col in ['user_age_level', 'user_occupation_id', 'user_star_level']:
        itemcnt = data.groupby([col, 'user_gender_id'], as_index=False)['instance_id'].agg({str(col) + '_user_gender_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'user_gender_id'], how='left')
        data[str(col) + '_user_gender_prob']=data[str(col) + '_user_gender_cnt']/data['user_gender_cnt']
    del data['user_gender_cnt']

    print('user_age_level对应的user_occupation_id，user_star_level')
    itemcnt = data.groupby(['user_age_level'], as_index=False)['instance_id'].agg({'user_age_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_age_level'], how='left')

    for col in ['user_occupation_id', 'user_star_level']:
        itemcnt = data.groupby([col, 'user_age_level'], as_index=False)['instance_id'].agg({str(col) + '_user_age_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'user_age_level'], how='left')
        data[str(col) + '_user_age_prob']=data[str(col) + '_user_age_cnt']/data['user_age_cnt']
    del data['user_age_cnt']

    print('user_occupation_id对应的user_star_level')
    itemcnt = data.groupby(['user_occupation_id'], as_index=False)['instance_id'].agg({'user_occ_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_occupation_id'], how='left')
    for col in ['user_star_level']:
        itemcnt = data.groupby([col, 'user_occupation_id'], as_index=False)['instance_id'].agg({str(col) + '_user_occ_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'user_occupation_id'], how='left')
        data[str(col) + '_user_occ_prob']=data[str(col) + '_user_occ_cnt']/data['user_occ_cnt']
    del data['user_occ_cnt']

    return data

def user_item(data):
    itemcnt = data.groupby(['user_id'], as_index=False)['instance_id'].agg({'user_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_id'], how='left')
    print('一个user有多少item_id,item_brand_id……')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = data.groupby([col, 'user_id'], as_index=False)['instance_id'].agg({str(col)+'_user_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_id'], how='left')
        data[str(col) + '_user_prob'] = data[str(col) + '_user_cnt'] / data['user_cnt']

    print('一个user_gender有多少item_id,item_brand_id……')
    itemcnt = data.groupby(['user_gender_id'], as_index=False)['instance_id'].agg({'user_gender_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_gender_id'], how='left')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = data.groupby([col, 'user_gender_id'], as_index=False)['instance_id'].agg({str(col)+'_user_gender_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_gender_id'], how='left')
        data[str(col) + '_user_gender_prob'] = data[str(col) + '_user_gender_cnt'] / data['user_gender_cnt']

    print('一个user_age_level有多少item_id,item_brand_id……')
    itemcnt = data.groupby(['user_age_level'], as_index=False)['instance_id'].agg({'user_age_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_age_level'], how='left')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = data.groupby([col, 'user_age_level'], as_index=False)['instance_id'].agg({str(col)+'_user_age_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_age_level'], how='left')
        data[str(col) + '_user_age_prob'] = data[str(col) + '_user_age_cnt'] / data['user_age_cnt']

    print('一个user_occupation_id有多少item_id,item_brand_id…')
    itemcnt = data.groupby(['user_occupation_id'], as_index=False)['instance_id'].agg({'user_occ_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_occupation_id'], how='left')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = data.groupby([col, 'user_occupation_id'], as_index=False)['instance_id'].agg({str(col)+'_user_occ_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_occupation_id'], how='left')
        data[str(col) + '_user_occ_prob'] = data[str(col) + '_user_occ_cnt'] / data['user_occ_cnt']

    return data


def user_shop(data):
    print('一个user有多少shop_id,shop_review_num_level……')

    for col in ['shop_id', 'shop_review_num_level', 'shop_star_level']:
        item_shop_cnt = data.groupby([col, 'user_id'], as_index=False)['instance_id'].agg(
            {str(col) + '_user_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_id'], how='left')
        data[str(col) + '_user_prob'] = data[str(col) + '_user_cnt'] / data['user_cnt']
    del data['user_cnt']

    print('一个user_gender有多少shop_id,shop_review_num_level……')
    for col in ['shop_id', 'shop_review_num_level', 'shop_star_level']:
        item_shop_cnt = data.groupby([col, 'user_gender_id'], as_index=False)['instance_id'].agg(
            {str(col) + '_user_gender_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_gender_id'], how='left')
        data[str(col) + '_user_gender_prob'] = data[str(col) + '_user_gender_cnt'] / data['user_gender_cnt']
    del data['user_gender_cnt']

    print('一个user_age_level有多少shop_id,shop_review_num_level……')
    for col in ['shop_id', 'shop_review_num_level', 'shop_star_level']:
        item_shop_cnt = data.groupby([col, 'user_age_level'], as_index=False)['instance_id'].agg(
            {str(col) + '_user_age_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_age_level'], how='left')
        data[str(col) + '_user_age_prob'] = data[str(col) + '_user_age_cnt'] / data['user_age_cnt']
    del data['user_age_cnt']

    print('一个user_occupation_id有多少shop_id,shop_review_num_level……')
    for col in ['shop_id', 'shop_review_num_level', 'shop_star_level']:
        item_shop_cnt = data.groupby([col, 'user_occupation_id'], as_index=False)['instance_id'].agg(
            {str(col) + '_user_occ_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_occupation_id'], how='left')
        data[str(col) + '_user_occ_prob'] = data[str(col) + '_user_occ_cnt'] / data['user_occ_cnt']
    del data['user_occ_cnt']

    return data


def shop_item(data):
    print('一个shop有多少item_id,item_brand_id,item_city_id,item_price_level……')
    itemcnt = data.groupby(['shop_id'], as_index=False)['instance_id'].agg({'shop_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['shop_id'], how='left')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = data.groupby([col, 'shop_id'], as_index=False)['instance_id'].agg({str(col)+'_shop_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'shop_id'], how='left')
        data[str(col) + '_shop_prob'] = data[str(col) + '_shop_cnt'] / data['shop_cnt']
    del data['shop_cnt']

    print('一个shop_review_num_level有多少item_id,item_brand_id,item_city_id,item_price_level……')
    itemcnt = data.groupby(['shop_review_num_level'], as_index=False)['instance_id'].agg({'shop_rev_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['shop_review_num_level'], how='left')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = data.groupby([col, 'shop_review_num_level'], as_index=False)['instance_id'].agg({str(col)+'_shop_rev_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'shop_review_num_level'], how='left')
        data[str(col) + '_shop_rev_prob'] = data[str(col) + '_shop_rev_cnt'] / data['shop_rev_cnt']
    del data['shop_rev_cnt']

    # print('一个shop_star_level有多少item_id,item_brand_id,item_city_id,item_price_level……')
    # itemcnt = data.groupby(['shop_star_level'], as_index=False)['instance_id'].agg({'shop_star_cnt': 'count'})
    # data = pd.merge(data, itemcnt, on=['shop_star_level'], how='left')
    # for col in ['item_id',
    #             'item_brand_id', 'item_city_id', 'item_price_level',
    #             'item_sales_level', 'item_collected_level', 'item_pv_level']:
    #     item_shop_cnt = data.groupby([col, 'shop_star_level'], as_index=False)['instance_id'].agg({str(col) + '_shop_star_cnt': 'count'})
    #     data = pd.merge(data, item_shop_cnt, on=[col, 'shop_star_level'], how='left')
    #     data[str(col) + '_shop_star_prob'] = data[str(col) + '_shop_star_cnt'] / data['shop_star_cnt']
    # del data['shop_star_cnt']
    return data

def ks_metric(true,score):
    fpr, tpr, thresholds = roc_curve(true,score)
    ks = max(tpr-fpr)
    return ks

In [20]:
def lgbCV(train, test,leaf=35):
    col = [c for c in train if
           c not in ['is_trade', 'item_category_list', 'item_property_list', 'predict_category_property', 'instance_id',
                     'context_id', 'realtime', 'context_timestamp']]
    # cat = ['sale_price', 'gender_star', 'user_age_level', 'item_price_level', 'item_sales_level', 'sale_collect',
    #        'price_collect', 'item_brand_id', 'user_star_level', 'item_id', 'shop_id',
    #        'item_city_id', 'context_page_id', 'gender_age', 'shop_star_level', 'item_pv_level', 'user_occupation_id',
    #        'day', 'gender_occ', 'user_gender_id']
    X = train[col]
    y = train['is_trade'].values
    X_tes = test[col]
    y_tes = test['is_trade'].values
    print('Training LGBM model...')
    lgb0 = lgb.LGBMClassifier(
        objective='binary',
        # metric='binary_error',
        num_leaves=leaf,
        depth=8,
        learning_rate=0.05,
        seed=2018,
        colsample_bytree=0.8,
        # min_child_samples=8,
        subsample=0.9,
        n_estimators=20000)
    lgb_model = lgb0.fit(X, y, eval_set=[(X_tes, y_tes)], early_stopping_rounds=200)
    best_iter = lgb_model.best_iteration_
    predictors = [i for i in X.columns]
    feat_imp = pd.Series(lgb_model.feature_importances_, predictors).sort_values(ascending=False)
    print(feat_imp)
    print(feat_imp.shape)
    # pred= lgb_model.predict(test[col])
    pred = lgb_model.predict_proba(test[col])[:, 1]
    test['pred'] = pred
    test['index'] = range(len(test))
    # print(test[['is_trade','pred']])
    print('误差 ', log_loss(test['is_trade'], test['pred']))
    print('KS ', ks_metric(test['is_trade'], test['pred']))
    logloss = log_loss(test['is_trade'], test['pred'])
    ks = ks_metric(test['is_trade'], test['pred'])
    return best_iter,lgb_model,pred

In [11]:
def sub(train, test, best_iter):
    col = [c for c in train if
           c not in ['is_trade', 'item_category_list', 'item_property_list', 'predict_category_property', 'instance_id',
                     'context_id', 'realtime', 'context_timestamp']]
    X = train[col]
    y = train['is_trade'].values
    print('Training LGBM model...')
    lgb0 = lgb.LGBMClassifier(
        objective='binary',
        # metric='binary_error',
        num_leaves=35,
        depth=8,
        learning_rate=0.05,
        seed=2018,
        colsample_bytree=0.8,
        # min_child_samples=8,
        subsample=0.9,
        n_estimators=best_iter)
    lgb_model = lgb0.fit(X, y)
    predictors = [i for i in X.columns]
    feat_imp = pd.Series(lgb_model.feature_importances_, predictors).sort_values(ascending=False)
    print(feat_imp)
    print(feat_imp.shape)
    # pred= lgb_model.predict(test[col])
    pred = lgb_model.predict_proba(test[col])[:, 1]
    test['predicted_score'] = pred
    sub1 = test[['instance_id', 'predicted_score']]
    sub=pd.read_csv("../../Data/advertisment/Raw/round1_ijcai_18_test_a_20180301.txt", sep="\s+")
    sub=pd.merge(sub,sub1,on=['instance_id'],how='left')
    sub=sub.fillna(0)
    #sub[['instance_id', 'predicted_score']].to_csv('result/result0320.csv',index=None,sep=' ')
    #sub[['instance_id', 'predicted_score']].to_csv('../../Submission/advertisement/result401.txt',sep=" ",index=False)
    return sub[['instance_id', 'predicted_score']]

In [23]:
def score_change(score,base_rate,real_rate):
    base_change = np.log(base_rate/(1-base_rate)) - np.log(real_rate/(1-real_rate))
    score_adj = np.exp(np.log(score/(1-score)) - base_change)/(np.exp(np.log(score/(1-score)) - base_change)+1)
    return score_adj

In [5]:
train = pd.read_csv("../../Data/advertisment/Raw/round1_ijcai_18_train_20180301.txt", sep="\s+")
test = pd.read_csv("../../Data/advertisment/Raw/round1_ijcai_18_test_a_20180301.txt", sep="\s+")
data = pd.concat([train, test])
data = data.drop_duplicates(subset='instance_id')  # 把instance id去重
print('make feature')
data = base_process(data)
data=shijian(data)
data=shop_fenduan(data)
data = slide_cnt(data)
data = zuhe(data)
print('----------------------------全局统计特征---------------------------------------------------')
data = item(data)
data = user(data)
data = user_item(data)
data = user_shop(data)
data=shop_item(data)

make feature
--------------------------------------------------------------item--------------------------------------------------------------
--------------------------------------------------------------user--------------------------------------------------------------
user 0,1 feature
--------------------------------------------------------------context--------------------------------------------------------------
context 0,1 feature
--------------------------------------------------------------shop--------------------------------------------------------------
3    493224
2      3146
1        25
Name: deliver_map, dtype: int64
3    490039
2      6243
1       113
Name: service_map, dtype: int64
3    394629
2     99788
1      1978
Name: de_map, dtype: int64
3    356965
2    138783
1       647
Name: review_map, dtype: int64
当前日期前一天的cnt
当前日期之前的cnt
前一个小时的统计量
item两两组合
user两两组合
shop两两组合
----------------------------全局统计特征---------------------------------------------------
一个item有多少brand,pric

In [55]:
train= data[(data['day'] >= 18) & (data['day'] <= 23)]
test= data[(data['day'] == 24)]

In [56]:
var = 'shop_id'
test_in = test.loc[test[var].isin(train[var].unique())]
test_out = test.loc[~test[var].isin(train[var].unique())]
print(test_in.shape)
print(test_in.is_trade.mean())
print(test_out.shape)
print(test_out.is_trade.mean())

(57136, 240)
0.016872024642957156
(266, 240)
0.015037593984962405


In [57]:
best_iter,model,pred = lgbCV(train,test_in,20)

Training LGBM model...
[1]	valid_0's binary_logloss: 0.647784
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.606714
[3]	valid_0's binary_logloss: 0.569409
[4]	valid_0's binary_logloss: 0.535377
[5]	valid_0's binary_logloss: 0.504208
[6]	valid_0's binary_logloss: 0.475592
[7]	valid_0's binary_logloss: 0.449259
[8]	valid_0's binary_logloss: 0.424972
[9]	valid_0's binary_logloss: 0.402516
[10]	valid_0's binary_logloss: 0.381719
[11]	valid_0's binary_logloss: 0.362417
[12]	valid_0's binary_logloss: 0.344482
[13]	valid_0's binary_logloss: 0.327809
[14]	valid_0's binary_logloss: 0.312285
[15]	valid_0's binary_logloss: 0.297802
[16]	valid_0's binary_logloss: 0.284298
[17]	valid_0's binary_logloss: 0.271693
[18]	valid_0's binary_logloss: 0.25989
[19]	valid_0's binary_logloss: 0.248863
[20]	valid_0's binary_logloss: 0.238543
[21]	valid_0's binary_logloss: 0.228874
[22]	valid_0's binary_logloss: 0.219815
[23]	valid_0's binary_logloss: 0.211327
[24]

[201]	valid_0's binary_logloss: 0.0810905
[202]	valid_0's binary_logloss: 0.0811004
[203]	valid_0's binary_logloss: 0.0811015
[204]	valid_0's binary_logloss: 0.0810877
[205]	valid_0's binary_logloss: 0.0810864
[206]	valid_0's binary_logloss: 0.0810792
[207]	valid_0's binary_logloss: 0.0810784
[208]	valid_0's binary_logloss: 0.0810664
[209]	valid_0's binary_logloss: 0.0810637
[210]	valid_0's binary_logloss: 0.0810696
[211]	valid_0's binary_logloss: 0.0810684
[212]	valid_0's binary_logloss: 0.0810695
[213]	valid_0's binary_logloss: 0.0810584
[214]	valid_0's binary_logloss: 0.0810395
[215]	valid_0's binary_logloss: 0.0810418
[216]	valid_0's binary_logloss: 0.0810377
[217]	valid_0's binary_logloss: 0.0810431
[218]	valid_0's binary_logloss: 0.081017
[219]	valid_0's binary_logloss: 0.0810173
[220]	valid_0's binary_logloss: 0.0810144
[221]	valid_0's binary_logloss: 0.0810183
[222]	valid_0's binary_logloss: 0.0810171
[223]	valid_0's binary_logloss: 0.0810086
[224]	valid_0's binary_logloss: 0.0

[398]	valid_0's binary_logloss: 0.0809001
[399]	valid_0's binary_logloss: 0.0808912
[400]	valid_0's binary_logloss: 0.0808929
[401]	valid_0's binary_logloss: 0.0808883
[402]	valid_0's binary_logloss: 0.0808822
[403]	valid_0's binary_logloss: 0.0808769
[404]	valid_0's binary_logloss: 0.0808786
[405]	valid_0's binary_logloss: 0.08087
[406]	valid_0's binary_logloss: 0.080873
[407]	valid_0's binary_logloss: 0.0808762
[408]	valid_0's binary_logloss: 0.0808821
[409]	valid_0's binary_logloss: 0.0808881
[410]	valid_0's binary_logloss: 0.0808949
[411]	valid_0's binary_logloss: 0.0809007
[412]	valid_0's binary_logloss: 0.0809024
[413]	valid_0's binary_logloss: 0.0809044
[414]	valid_0's binary_logloss: 0.0809081
[415]	valid_0's binary_logloss: 0.0809109
[416]	valid_0's binary_logloss: 0.0809022
[417]	valid_0's binary_logloss: 0.0809019
[418]	valid_0's binary_logloss: 0.0809067
[419]	valid_0's binary_logloss: 0.0808989
[420]	valid_0's binary_logloss: 0.0808991
[421]	valid_0's binary_logloss: 0.080

In [58]:
best_iter,model,pred = lgbCV(train,test_out,20)

Training LGBM model...
[1]	valid_0's binary_logloss: 0.647671
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.606678
[3]	valid_0's binary_logloss: 0.569405
[4]	valid_0's binary_logloss: 0.535393
[5]	valid_0's binary_logloss: 0.504223
[6]	valid_0's binary_logloss: 0.475545
[7]	valid_0's binary_logloss: 0.449176
[8]	valid_0's binary_logloss: 0.42484
[9]	valid_0's binary_logloss: 0.402316
[10]	valid_0's binary_logloss: 0.381394
[11]	valid_0's binary_logloss: 0.361917
[12]	valid_0's binary_logloss: 0.343863
[13]	valid_0's binary_logloss: 0.327138
[14]	valid_0's binary_logloss: 0.311519
[15]	valid_0's binary_logloss: 0.296871
[16]	valid_0's binary_logloss: 0.283324
[17]	valid_0's binary_logloss: 0.270663
[18]	valid_0's binary_logloss: 0.258919
[19]	valid_0's binary_logloss: 0.247782
[20]	valid_0's binary_logloss: 0.237377
[21]	valid_0's binary_logloss: 0.227765
[22]	valid_0's binary_logloss: 0.218606
[23]	valid_0's binary_logloss: 0.210027
[24]

[201]	valid_0's binary_logloss: 0.0832615
[202]	valid_0's binary_logloss: 0.0832789
[203]	valid_0's binary_logloss: 0.0833646
[204]	valid_0's binary_logloss: 0.0834844
[205]	valid_0's binary_logloss: 0.083496
[206]	valid_0's binary_logloss: 0.0834961
[207]	valid_0's binary_logloss: 0.0835066
[208]	valid_0's binary_logloss: 0.083496
[209]	valid_0's binary_logloss: 0.0835438
[210]	valid_0's binary_logloss: 0.083616
[211]	valid_0's binary_logloss: 0.0836997
[212]	valid_0's binary_logloss: 0.0837412
[213]	valid_0's binary_logloss: 0.0837938
[214]	valid_0's binary_logloss: 0.0838099
[215]	valid_0's binary_logloss: 0.0838148
[216]	valid_0's binary_logloss: 0.0837784
[217]	valid_0's binary_logloss: 0.0837015
[218]	valid_0's binary_logloss: 0.0837152
[219]	valid_0's binary_logloss: 0.0838395
[220]	valid_0's binary_logloss: 0.083933
[221]	valid_0's binary_logloss: 0.084012
[222]	valid_0's binary_logloss: 0.0840221
[223]	valid_0's binary_logloss: 0.0840194
[224]	valid_0's binary_logloss: 0.08408

In [59]:
del train[var]
del test_in[var]
del test_out[var]

In [60]:
best_iter,model,pred = lgbCV(train,test_in,20)

Training LGBM model...
[1]	valid_0's binary_logloss: 0.647794
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.60673
[3]	valid_0's binary_logloss: 0.569421
[4]	valid_0's binary_logloss: 0.535383
[5]	valid_0's binary_logloss: 0.504209
[6]	valid_0's binary_logloss: 0.475596
[7]	valid_0's binary_logloss: 0.449246
[8]	valid_0's binary_logloss: 0.424957
[9]	valid_0's binary_logloss: 0.402498
[10]	valid_0's binary_logloss: 0.381704
[11]	valid_0's binary_logloss: 0.362407
[12]	valid_0's binary_logloss: 0.344479
[13]	valid_0's binary_logloss: 0.327798
[14]	valid_0's binary_logloss: 0.31227
[15]	valid_0's binary_logloss: 0.297792
[16]	valid_0's binary_logloss: 0.284281
[17]	valid_0's binary_logloss: 0.271675
[18]	valid_0's binary_logloss: 0.259879
[19]	valid_0's binary_logloss: 0.248848
[20]	valid_0's binary_logloss: 0.238518
[21]	valid_0's binary_logloss: 0.228845
[22]	valid_0's binary_logloss: 0.219792
[23]	valid_0's binary_logloss: 0.211307
[24]	

[201]	valid_0's binary_logloss: 0.0811611
[202]	valid_0's binary_logloss: 0.0811611
[203]	valid_0's binary_logloss: 0.0811629
[204]	valid_0's binary_logloss: 0.0811483
[205]	valid_0's binary_logloss: 0.0811423
[206]	valid_0's binary_logloss: 0.0811317
[207]	valid_0's binary_logloss: 0.0811234
[208]	valid_0's binary_logloss: 0.0811089
[209]	valid_0's binary_logloss: 0.0811178
[210]	valid_0's binary_logloss: 0.0811031
[211]	valid_0's binary_logloss: 0.0811038
[212]	valid_0's binary_logloss: 0.0811099
[213]	valid_0's binary_logloss: 0.0811114
[214]	valid_0's binary_logloss: 0.0811022
[215]	valid_0's binary_logloss: 0.081094
[216]	valid_0's binary_logloss: 0.0810911
[217]	valid_0's binary_logloss: 0.0810947
[218]	valid_0's binary_logloss: 0.0810863
[219]	valid_0's binary_logloss: 0.0810903
[220]	valid_0's binary_logloss: 0.0810913
[221]	valid_0's binary_logloss: 0.0810941
[222]	valid_0's binary_logloss: 0.081093
[223]	valid_0's binary_logloss: 0.0810968
[224]	valid_0's binary_logloss: 0.08

[397]	valid_0's binary_logloss: 0.081154
[398]	valid_0's binary_logloss: 0.0811565
[399]	valid_0's binary_logloss: 0.0811593
[400]	valid_0's binary_logloss: 0.0811539
[401]	valid_0's binary_logloss: 0.0811564
[402]	valid_0's binary_logloss: 0.0811567
[403]	valid_0's binary_logloss: 0.0811605
[404]	valid_0's binary_logloss: 0.0811641
[405]	valid_0's binary_logloss: 0.0811627
[406]	valid_0's binary_logloss: 0.0811638
[407]	valid_0's binary_logloss: 0.0811579
[408]	valid_0's binary_logloss: 0.0811635
[409]	valid_0's binary_logloss: 0.0811663
[410]	valid_0's binary_logloss: 0.0811701
[411]	valid_0's binary_logloss: 0.0811724
[412]	valid_0's binary_logloss: 0.0811791
[413]	valid_0's binary_logloss: 0.0811827
[414]	valid_0's binary_logloss: 0.0811815
[415]	valid_0's binary_logloss: 0.0811913
[416]	valid_0's binary_logloss: 0.0811904
[417]	valid_0's binary_logloss: 0.0811923
[418]	valid_0's binary_logloss: 0.0812025
[419]	valid_0's binary_logloss: 0.0812041
[420]	valid_0's binary_logloss: 0.0

In [61]:
best_iter,model,pred = lgbCV(train,test_out,20)

Training LGBM model...
[1]	valid_0's binary_logloss: 0.64781
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.606864
[3]	valid_0's binary_logloss: 0.569427
[4]	valid_0's binary_logloss: 0.535404
[5]	valid_0's binary_logloss: 0.504189
[6]	valid_0's binary_logloss: 0.475464
[7]	valid_0's binary_logloss: 0.449074
[8]	valid_0's binary_logloss: 0.424675
[9]	valid_0's binary_logloss: 0.402295
[10]	valid_0's binary_logloss: 0.381343
[11]	valid_0's binary_logloss: 0.361903
[12]	valid_0's binary_logloss: 0.343865
[13]	valid_0's binary_logloss: 0.327208
[14]	valid_0's binary_logloss: 0.311537
[15]	valid_0's binary_logloss: 0.296972
[16]	valid_0's binary_logloss: 0.283413
[17]	valid_0's binary_logloss: 0.270748
[18]	valid_0's binary_logloss: 0.258828
[19]	valid_0's binary_logloss: 0.247683
[20]	valid_0's binary_logloss: 0.237341
[21]	valid_0's binary_logloss: 0.227577
[22]	valid_0's binary_logloss: 0.218516
[23]	valid_0's binary_logloss: 0.209903
[24]

[200]	valid_0's binary_logloss: 0.0826612
[201]	valid_0's binary_logloss: 0.0826717
[202]	valid_0's binary_logloss: 0.0827152
[203]	valid_0's binary_logloss: 0.0810376
[204]	valid_0's binary_logloss: 0.0810784
[205]	valid_0's binary_logloss: 0.0810794
[206]	valid_0's binary_logloss: 0.0812095
[207]	valid_0's binary_logloss: 0.0812204
[208]	valid_0's binary_logloss: 0.0812981
[209]	valid_0's binary_logloss: 0.0813121
[210]	valid_0's binary_logloss: 0.0814243
[211]	valid_0's binary_logloss: 0.0813047
[212]	valid_0's binary_logloss: 0.0814211
[213]	valid_0's binary_logloss: 0.0814192
[214]	valid_0's binary_logloss: 0.0814169
[215]	valid_0's binary_logloss: 0.0814338
[216]	valid_0's binary_logloss: 0.0814534
[217]	valid_0's binary_logloss: 0.0814332
[218]	valid_0's binary_logloss: 0.0814236
[219]	valid_0's binary_logloss: 0.0815039
[220]	valid_0's binary_logloss: 0.0815058
[221]	valid_0's binary_logloss: 0.081628
[222]	valid_0's binary_logloss: 0.081624
[223]	valid_0's binary_logloss: 0.08

In [21]:
best_iter,model,pred = lgbCV(train,test,20)

Training LGBM model...
[1]	valid_0's binary_logloss: 0.647784
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.606714
[3]	valid_0's binary_logloss: 0.569409
[4]	valid_0's binary_logloss: 0.535377
[5]	valid_0's binary_logloss: 0.504208
[6]	valid_0's binary_logloss: 0.475592
[7]	valid_0's binary_logloss: 0.449259
[8]	valid_0's binary_logloss: 0.424972
[9]	valid_0's binary_logloss: 0.402515
[10]	valid_0's binary_logloss: 0.381717
[11]	valid_0's binary_logloss: 0.362415
[12]	valid_0's binary_logloss: 0.344479
[13]	valid_0's binary_logloss: 0.327806
[14]	valid_0's binary_logloss: 0.312282
[15]	valid_0's binary_logloss: 0.297797
[16]	valid_0's binary_logloss: 0.284294
[17]	valid_0's binary_logloss: 0.271688
[18]	valid_0's binary_logloss: 0.259886
[19]	valid_0's binary_logloss: 0.248858
[20]	valid_0's binary_logloss: 0.238537
[21]	valid_0's binary_logloss: 0.228869
[22]	valid_0's binary_logloss: 0.21981
[23]	valid_0's binary_logloss: 0.211321
[24]

[201]	valid_0's binary_logloss: 0.0811005
[202]	valid_0's binary_logloss: 0.0811105
[203]	valid_0's binary_logloss: 0.081112
[204]	valid_0's binary_logloss: 0.0810988
[205]	valid_0's binary_logloss: 0.0810976
[206]	valid_0's binary_logloss: 0.0810904
[207]	valid_0's binary_logloss: 0.0810897
[208]	valid_0's binary_logloss: 0.0810777
[209]	valid_0's binary_logloss: 0.0810752
[210]	valid_0's binary_logloss: 0.0810814
[211]	valid_0's binary_logloss: 0.0810805
[212]	valid_0's binary_logloss: 0.0810819
[213]	valid_0's binary_logloss: 0.0810711
[214]	valid_0's binary_logloss: 0.0810523
[215]	valid_0's binary_logloss: 0.0810546
[216]	valid_0's binary_logloss: 0.0810504
[217]	valid_0's binary_logloss: 0.0810554
[218]	valid_0's binary_logloss: 0.0810295
[219]	valid_0's binary_logloss: 0.0810303
[220]	valid_0's binary_logloss: 0.0810279
[221]	valid_0's binary_logloss: 0.0810322
[222]	valid_0's binary_logloss: 0.081031
[223]	valid_0's binary_logloss: 0.0810226
[224]	valid_0's binary_logloss: 0.08

[399]	valid_0's binary_logloss: 0.080913
[400]	valid_0's binary_logloss: 0.0809146
[401]	valid_0's binary_logloss: 0.08091
[402]	valid_0's binary_logloss: 0.0809041
[403]	valid_0's binary_logloss: 0.0808987
[404]	valid_0's binary_logloss: 0.0809007
[405]	valid_0's binary_logloss: 0.080893
[406]	valid_0's binary_logloss: 0.0808963
[407]	valid_0's binary_logloss: 0.0808994
[408]	valid_0's binary_logloss: 0.0809049
[409]	valid_0's binary_logloss: 0.0809109
[410]	valid_0's binary_logloss: 0.0809179
[411]	valid_0's binary_logloss: 0.0809236
[412]	valid_0's binary_logloss: 0.0809255
[413]	valid_0's binary_logloss: 0.0809275
[414]	valid_0's binary_logloss: 0.0809311
[415]	valid_0's binary_logloss: 0.0809344
[416]	valid_0's binary_logloss: 0.0809263
[417]	valid_0's binary_logloss: 0.0809259
[418]	valid_0's binary_logloss: 0.0809307
[419]	valid_0's binary_logloss: 0.0809232
[420]	valid_0's binary_logloss: 0.0809241
[421]	valid_0's binary_logloss: 0.0809079
[422]	valid_0's binary_logloss: 0.0809

In [24]:
pred_adj = score_change(pred,train.is_trade.mean(),test.is_trade.mean())
print(log_loss(test['is_trade'], pred_adj))

0.0807860876327


In [32]:
train = data[data.is_trade.notnull()]
test = data[data.is_trade.isnull()]
xx = sub(train, test, best_iter)

Training LGBM model...
shop_score_delivery                    249
user_id                                214
predict_category_property0             193
item_id_user_prob                      176
hour                                   162
predict_category_property3             142
predict_category_property2             135
predict_category_property1             133
user_star_level_user_age_prob          132
item_brand_id_user_prob                124
gender_star                            124
item_collected_level_salse_prob        121
user_cntx                              115
item_price_level                       115
shop_cntx                              112
sale_price                             106
hour_map                               105
len_item_property                      104
shop_score_service                      99
shop_cnt1                               98
item_cntx                               96
item_price_level_user_age_prob          95
predict_category_property4     

In [34]:
xx['predicted_score'] = score_change(xx['predicted_score'],xx['predicted_score'].mean(),0.0169830864007192)

In [35]:
xx['predicted_score'].mean()

0.017027874765400058

In [36]:
xx.to_csv('../../Submission/advertisement/result410.txt',sep=" ",index=False)

In [15]:
del train['user_id']
del test['user_id']

In [6]:
tmp = train.loc[train['is_trade']==1]
train_new = pd.concat([train,tmp])

In [14]:
loglossList =[]
ksList = []
for leaf in range(5,61,5):
    best_iter,model,logloss,ks = lgbCV(train, test,leaf)
    loglossList.append(logloss)
    ksList.append(ks)

Training LGBM model...
[1]	valid_0's binary_logloss: 0.647801
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.606755
[3]	valid_0's binary_logloss: 0.569466
[4]	valid_0's binary_logloss: 0.53544
[5]	valid_0's binary_logloss: 0.504286
[6]	valid_0's binary_logloss: 0.475681
[7]	valid_0's binary_logloss: 0.449347
[8]	valid_0's binary_logloss: 0.425063
[9]	valid_0's binary_logloss: 0.402621
[10]	valid_0's binary_logloss: 0.381826
[11]	valid_0's binary_logloss: 0.362549
[12]	valid_0's binary_logloss: 0.344628
[13]	valid_0's binary_logloss: 0.327963
[14]	valid_0's binary_logloss: 0.312446
[15]	valid_0's binary_logloss: 0.29797
[16]	valid_0's binary_logloss: 0.284469
[17]	valid_0's binary_logloss: 0.271869
[18]	valid_0's binary_logloss: 0.26008
[19]	valid_0's binary_logloss: 0.249064
[20]	valid_0's binary_logloss: 0.23875
[21]	valid_0's binary_logloss: 0.229091
[22]	valid_0's binary_logloss: 0.220046
[23]	valid_0's binary_logloss: 0.211569
[24]	va

[200]	valid_0's binary_logloss: 0.0815244
[201]	valid_0's binary_logloss: 0.0815212
[202]	valid_0's binary_logloss: 0.0815225
[203]	valid_0's binary_logloss: 0.0815228
[204]	valid_0's binary_logloss: 0.0815157
[205]	valid_0's binary_logloss: 0.0815102
[206]	valid_0's binary_logloss: 0.0815077
[207]	valid_0's binary_logloss: 0.0815033
[208]	valid_0's binary_logloss: 0.0814956
[209]	valid_0's binary_logloss: 0.0814904
[210]	valid_0's binary_logloss: 0.0814857
[211]	valid_0's binary_logloss: 0.0814799
[212]	valid_0's binary_logloss: 0.0814746
[213]	valid_0's binary_logloss: 0.0814725
[214]	valid_0's binary_logloss: 0.0814629
[215]	valid_0's binary_logloss: 0.0814584
[216]	valid_0's binary_logloss: 0.0814533
[217]	valid_0's binary_logloss: 0.0814548
[218]	valid_0's binary_logloss: 0.0814444
[219]	valid_0's binary_logloss: 0.0814358
[220]	valid_0's binary_logloss: 0.0814257
[221]	valid_0's binary_logloss: 0.0814252
[222]	valid_0's binary_logloss: 0.0814219
[223]	valid_0's binary_logloss: 0.

[398]	valid_0's binary_logloss: 0.0811802
[399]	valid_0's binary_logloss: 0.0811797
[400]	valid_0's binary_logloss: 0.0811784
[401]	valid_0's binary_logloss: 0.0811747
[402]	valid_0's binary_logloss: 0.0811692
[403]	valid_0's binary_logloss: 0.0811669
[404]	valid_0's binary_logloss: 0.0811672
[405]	valid_0's binary_logloss: 0.0811599
[406]	valid_0's binary_logloss: 0.0811631
[407]	valid_0's binary_logloss: 0.0811637
[408]	valid_0's binary_logloss: 0.0811654
[409]	valid_0's binary_logloss: 0.0811607
[410]	valid_0's binary_logloss: 0.0811618
[411]	valid_0's binary_logloss: 0.0811625
[412]	valid_0's binary_logloss: 0.08116
[413]	valid_0's binary_logloss: 0.0811624
[414]	valid_0's binary_logloss: 0.0811652
[415]	valid_0's binary_logloss: 0.0811637
[416]	valid_0's binary_logloss: 0.0811631
[417]	valid_0's binary_logloss: 0.081164
[418]	valid_0's binary_logloss: 0.0811645
[419]	valid_0's binary_logloss: 0.0811599
[420]	valid_0's binary_logloss: 0.0811535
[421]	valid_0's binary_logloss: 0.081

[595]	valid_0's binary_logloss: 0.0810794
[596]	valid_0's binary_logloss: 0.0810822
[597]	valid_0's binary_logloss: 0.081083
[598]	valid_0's binary_logloss: 0.0810851
[599]	valid_0's binary_logloss: 0.0810835
[600]	valid_0's binary_logloss: 0.0810859
[601]	valid_0's binary_logloss: 0.0810905
[602]	valid_0's binary_logloss: 0.0810916
[603]	valid_0's binary_logloss: 0.0810919
[604]	valid_0's binary_logloss: 0.0810933
[605]	valid_0's binary_logloss: 0.081097
[606]	valid_0's binary_logloss: 0.0810923
[607]	valid_0's binary_logloss: 0.0810931
[608]	valid_0's binary_logloss: 0.0810913
[609]	valid_0's binary_logloss: 0.0810911
[610]	valid_0's binary_logloss: 0.0810934
[611]	valid_0's binary_logloss: 0.0810978
[612]	valid_0's binary_logloss: 0.0810968
[613]	valid_0's binary_logloss: 0.0810965
[614]	valid_0's binary_logloss: 0.081095
[615]	valid_0's binary_logloss: 0.0810967
[616]	valid_0's binary_logloss: 0.0810946
[617]	valid_0's binary_logloss: 0.0810954
[618]	valid_0's binary_logloss: 0.081

[793]	valid_0's binary_logloss: 0.0810382
[794]	valid_0's binary_logloss: 0.0810358
[795]	valid_0's binary_logloss: 0.0810396
[796]	valid_0's binary_logloss: 0.0810399
[797]	valid_0's binary_logloss: 0.0810409
[798]	valid_0's binary_logloss: 0.0810419
[799]	valid_0's binary_logloss: 0.0810473
[800]	valid_0's binary_logloss: 0.0810469
[801]	valid_0's binary_logloss: 0.0810457
[802]	valid_0's binary_logloss: 0.0810447
[803]	valid_0's binary_logloss: 0.0810441
[804]	valid_0's binary_logloss: 0.0810453
[805]	valid_0's binary_logloss: 0.081047
[806]	valid_0's binary_logloss: 0.0810481
[807]	valid_0's binary_logloss: 0.0810494
[808]	valid_0's binary_logloss: 0.0810487
[809]	valid_0's binary_logloss: 0.0810504
[810]	valid_0's binary_logloss: 0.0810503
[811]	valid_0's binary_logloss: 0.0810496
[812]	valid_0's binary_logloss: 0.0810479
[813]	valid_0's binary_logloss: 0.0810457
[814]	valid_0's binary_logloss: 0.0810465
[815]	valid_0's binary_logloss: 0.081049
[816]	valid_0's binary_logloss: 0.08

...

误差  0.0811103598342
KS  0.292898608687
Training LGBM model...
[1]	valid_0's binary_logloss: 0.647781
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.606705
[3]	valid_0's binary_logloss: 0.569387
[4]	valid_0's binary_logloss: 0.535354
[5]	valid_0's binary_logloss: 0.504171
[6]	valid_0's binary_logloss: 0.475562
[7]	valid_0's binary_logloss: 0.449222
[8]	valid_0's binary_logloss: 0.424935
[9]	valid_0's binary_logloss: 0.402477
[10]	valid_0's binary_logloss: 0.381681
[11]	valid_0's binary_logloss: 0.36239
[12]	valid_0's binary_logloss: 0.34445
[13]	valid_0's binary_logloss: 0.327759
[14]	valid_0's binary_logloss: 0.312227
[15]	valid_0's binary_logloss: 0.297747
[16]	valid_0's binary_logloss: 0.284235
[17]	valid_0's binary_logloss: 0.271627
[18]	valid_0's binary_logloss: 0.25982
[19]	valid_0's binary_logloss: 0.248788
[20]	valid_0's binary_logloss: 0.238465
[21]	valid_0's binary_logloss: 0.228787
[22]	valid_0's binary_logloss: 0.219729
[23]	va

[198]	valid_0's binary_logloss: 0.0812634
[199]	valid_0's binary_logloss: 0.0812687
[200]	valid_0's binary_logloss: 0.0812589
[201]	valid_0's binary_logloss: 0.0812431
[202]	valid_0's binary_logloss: 0.081241
[203]	valid_0's binary_logloss: 0.081257
[204]	valid_0's binary_logloss: 0.0812664
[205]	valid_0's binary_logloss: 0.0812606
[206]	valid_0's binary_logloss: 0.0812577
[207]	valid_0's binary_logloss: 0.0812665
[208]	valid_0's binary_logloss: 0.0812658
[209]	valid_0's binary_logloss: 0.0812714
[210]	valid_0's binary_logloss: 0.0812821
[211]	valid_0's binary_logloss: 0.0812716
[212]	valid_0's binary_logloss: 0.0812667
[213]	valid_0's binary_logloss: 0.0812628
[214]	valid_0's binary_logloss: 0.0812569
[215]	valid_0's binary_logloss: 0.0812643
[216]	valid_0's binary_logloss: 0.0812621
[217]	valid_0's binary_logloss: 0.0812666
[218]	valid_0's binary_logloss: 0.0812782
[219]	valid_0's binary_logloss: 0.0812891
[220]	valid_0's binary_logloss: 0.0812988
[221]	valid_0's binary_logloss: 0.08

误差  0.081186197568
KS  0.297385529267
Training LGBM model...
[1]	valid_0's binary_logloss: 0.647784
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.606708
[3]	valid_0's binary_logloss: 0.569386
[4]	valid_0's binary_logloss: 0.535356
[5]	valid_0's binary_logloss: 0.504175
[6]	valid_0's binary_logloss: 0.475565
[7]	valid_0's binary_logloss: 0.449223
[8]	valid_0's binary_logloss: 0.424946
[9]	valid_0's binary_logloss: 0.402485
[10]	valid_0's binary_logloss: 0.381682
[11]	valid_0's binary_logloss: 0.362389
[12]	valid_0's binary_logloss: 0.344449
[13]	valid_0's binary_logloss: 0.327772
[14]	valid_0's binary_logloss: 0.312241
[15]	valid_0's binary_logloss: 0.29776
[16]	valid_0's binary_logloss: 0.284244
[17]	valid_0's binary_logloss: 0.271635
[18]	valid_0's binary_logloss: 0.259836
[19]	valid_0's binary_logloss: 0.248806
[20]	valid_0's binary_logloss: 0.238482
[21]	valid_0's binary_logloss: 0.228844
[22]	valid_0's binary_logloss: 0.219781
[23]	v

[198]	valid_0's binary_logloss: 0.0813883
[199]	valid_0's binary_logloss: 0.0813943
[200]	valid_0's binary_logloss: 0.0813961
[201]	valid_0's binary_logloss: 0.0813868
[202]	valid_0's binary_logloss: 0.0813812
[203]	valid_0's binary_logloss: 0.0813887
[204]	valid_0's binary_logloss: 0.0813898
[205]	valid_0's binary_logloss: 0.0813958
[206]	valid_0's binary_logloss: 0.0814031
[207]	valid_0's binary_logloss: 0.0814068
[208]	valid_0's binary_logloss: 0.0814137
[209]	valid_0's binary_logloss: 0.081421
[210]	valid_0's binary_logloss: 0.0814113
[211]	valid_0's binary_logloss: 0.0814146
[212]	valid_0's binary_logloss: 0.0814232
[213]	valid_0's binary_logloss: 0.0814206
[214]	valid_0's binary_logloss: 0.081405
[215]	valid_0's binary_logloss: 0.0814086
[216]	valid_0's binary_logloss: 0.0814142
[217]	valid_0's binary_logloss: 0.0814235
[218]	valid_0's binary_logloss: 0.0814308
[219]	valid_0's binary_logloss: 0.0814267
[220]	valid_0's binary_logloss: 0.0814224
[221]	valid_0's binary_logloss: 0.08

误差  0.0812065342325
KS  0.291527373306


In [17]:
tmp = train.loc[train['is_trade']==1]
train_new = pd.concat([train,tmp])
best_iter,model,logloss,ks = lgbCV(train_new, test,20)

Training LGBM model...
[1]	valid_0's binary_logloss: 0.649401
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.609733
[3]	valid_0's binary_logloss: 0.573641
[4]	valid_0's binary_logloss: 0.540653
[5]	valid_0's binary_logloss: 0.510406
[6]	valid_0's binary_logloss: 0.482624
[7]	valid_0's binary_logloss: 0.456997
[8]	valid_0's binary_logloss: 0.433334
[9]	valid_0's binary_logloss: 0.411457
[10]	valid_0's binary_logloss: 0.391148
[11]	valid_0's binary_logloss: 0.372322
[12]	valid_0's binary_logloss: 0.354827
[13]	valid_0's binary_logloss: 0.338504
[14]	valid_0's binary_logloss: 0.323323
[15]	valid_0's binary_logloss: 0.30915
[16]	valid_0's binary_logloss: 0.295885
[17]	valid_0's binary_logloss: 0.283504
[18]	valid_0's binary_logloss: 0.271922
[19]	valid_0's binary_logloss: 0.261096
[20]	valid_0's binary_logloss: 0.250954
[21]	valid_0's binary_logloss: 0.241439
[22]	valid_0's binary_logloss: 0.232501
[23]	valid_0's binary_logloss: 0.224116
[24]

[200]	valid_0's binary_logloss: 0.0874309
[201]	valid_0's binary_logloss: 0.0874271
[202]	valid_0's binary_logloss: 0.0874364
[203]	valid_0's binary_logloss: 0.0874289
[204]	valid_0's binary_logloss: 0.0874257
[205]	valid_0's binary_logloss: 0.0874245
[206]	valid_0's binary_logloss: 0.0874158
[207]	valid_0's binary_logloss: 0.0874133
[208]	valid_0's binary_logloss: 0.0874112
[209]	valid_0's binary_logloss: 0.0874179
[210]	valid_0's binary_logloss: 0.0874182
[211]	valid_0's binary_logloss: 0.0874084
[212]	valid_0's binary_logloss: 0.0873239
[213]	valid_0's binary_logloss: 0.0873266
[214]	valid_0's binary_logloss: 0.0873174
[215]	valid_0's binary_logloss: 0.0873207
[216]	valid_0's binary_logloss: 0.0873263
[217]	valid_0's binary_logloss: 0.0873363
[218]	valid_0's binary_logloss: 0.0873351
[219]	valid_0's binary_logloss: 0.0873326
[220]	valid_0's binary_logloss: 0.0873399
[221]	valid_0's binary_logloss: 0.087338
[222]	valid_0's binary_logloss: 0.0873427
[223]	valid_0's binary_logloss: 0.0

[396]	valid_0's binary_logloss: 0.0868389
[397]	valid_0's binary_logloss: 0.0868436
[398]	valid_0's binary_logloss: 0.0868436
[399]	valid_0's binary_logloss: 0.0868416
[400]	valid_0's binary_logloss: 0.0868322
[401]	valid_0's binary_logloss: 0.0868284
[402]	valid_0's binary_logloss: 0.0868238
[403]	valid_0's binary_logloss: 0.0868255
[404]	valid_0's binary_logloss: 0.0868225
[405]	valid_0's binary_logloss: 0.0868208
[406]	valid_0's binary_logloss: 0.0868097
[407]	valid_0's binary_logloss: 0.0868075
[408]	valid_0's binary_logloss: 0.0868071
[409]	valid_0's binary_logloss: 0.0868092
[410]	valid_0's binary_logloss: 0.086815
[411]	valid_0's binary_logloss: 0.086817
[412]	valid_0's binary_logloss: 0.0868138
[413]	valid_0's binary_logloss: 0.0868185
[414]	valid_0's binary_logloss: 0.0868147
[415]	valid_0's binary_logloss: 0.0868103
[416]	valid_0's binary_logloss: 0.0868069
[417]	valid_0's binary_logloss: 0.0867987
[418]	valid_0's binary_logloss: 0.0868019
[419]	valid_0's binary_logloss: 0.08

[593]	valid_0's binary_logloss: 0.0863778
[594]	valid_0's binary_logloss: 0.0863785
[595]	valid_0's binary_logloss: 0.0863815
[596]	valid_0's binary_logloss: 0.0863771
[597]	valid_0's binary_logloss: 0.0863745
[598]	valid_0's binary_logloss: 0.0863766
[599]	valid_0's binary_logloss: 0.0863763
[600]	valid_0's binary_logloss: 0.0863701
[601]	valid_0's binary_logloss: 0.0863707
[602]	valid_0's binary_logloss: 0.0863678
[603]	valid_0's binary_logloss: 0.0863656
[604]	valid_0's binary_logloss: 0.0863676
[605]	valid_0's binary_logloss: 0.0863639
[606]	valid_0's binary_logloss: 0.0863594
[607]	valid_0's binary_logloss: 0.0863548
[608]	valid_0's binary_logloss: 0.086344
[609]	valid_0's binary_logloss: 0.086339
[610]	valid_0's binary_logloss: 0.0863359
[611]	valid_0's binary_logloss: 0.0863302
[612]	valid_0's binary_logloss: 0.0863207
[613]	valid_0's binary_logloss: 0.0863086
[614]	valid_0's binary_logloss: 0.0863054
[615]	valid_0's binary_logloss: 0.0862917
[616]	valid_0's binary_logloss: 0.08

[789]	valid_0's binary_logloss: 0.0859507
[790]	valid_0's binary_logloss: 0.0859479
[791]	valid_0's binary_logloss: 0.0859453
[792]	valid_0's binary_logloss: 0.0859408
[793]	valid_0's binary_logloss: 0.085945
[794]	valid_0's binary_logloss: 0.0859388
[795]	valid_0's binary_logloss: 0.0859373
[796]	valid_0's binary_logloss: 0.0859372
[797]	valid_0's binary_logloss: 0.0859388
[798]	valid_0's binary_logloss: 0.0859385
[799]	valid_0's binary_logloss: 0.0859464
[800]	valid_0's binary_logloss: 0.0859452
[801]	valid_0's binary_logloss: 0.0859165
[802]	valid_0's binary_logloss: 0.0859151
[803]	valid_0's binary_logloss: 0.0859162
[804]	valid_0's binary_logloss: 0.0859167
[805]	valid_0's binary_logloss: 0.0859156
[806]	valid_0's binary_logloss: 0.0859202
[807]	valid_0's binary_logloss: 0.0859209
[808]	valid_0's binary_logloss: 0.0859285
[809]	valid_0's binary_logloss: 0.0859254
[810]	valid_0's binary_logloss: 0.0859226
[811]	valid_0's binary_logloss: 0.085919
[812]	valid_0's binary_logloss: 0.08

...

[2523]	valid_0's binary_logloss: 0.084236
[2524]	valid_0's binary_logloss: 0.0842343
[2525]	valid_0's binary_logloss: 0.0842252
[2526]	valid_0's binary_logloss: 0.084223
[2527]	valid_0's binary_logloss: 0.0842221
[2528]	valid_0's binary_logloss: 0.0842221
[2529]	valid_0's binary_logloss: 0.0842203
[2530]	valid_0's binary_logloss: 0.0842199
[2531]	valid_0's binary_logloss: 0.0842184
[2532]	valid_0's binary_logloss: 0.0842133
[2533]	valid_0's binary_logloss: 0.084216
[2534]	valid_0's binary_logloss: 0.0842159
[2535]	valid_0's binary_logloss: 0.0842049
[2536]	valid_0's binary_logloss: 0.0842077
[2537]	valid_0's binary_logloss: 0.0842122
[2538]	valid_0's binary_logloss: 0.0842125
[2539]	valid_0's binary_logloss: 0.0842118
[2540]	valid_0's binary_logloss: 0.0842053
[2541]	valid_0's binary_logloss: 0.0842031
[2542]	valid_0's binary_logloss: 0.0842044
[2543]	valid_0's binary_logloss: 0.0842092
[2544]	valid_0's binary_logloss: 0.0842126
[2545]	valid_0's binary_logloss: 0.084213
[2546]	valid_0'

[2715]	valid_0's binary_logloss: 0.0841878
[2716]	valid_0's binary_logloss: 0.0841884
[2717]	valid_0's binary_logloss: 0.084189
[2718]	valid_0's binary_logloss: 0.0841838
[2719]	valid_0's binary_logloss: 0.0841866
[2720]	valid_0's binary_logloss: 0.0841914
[2721]	valid_0's binary_logloss: 0.0841911
[2722]	valid_0's binary_logloss: 0.0841877
[2723]	valid_0's binary_logloss: 0.084188
[2724]	valid_0's binary_logloss: 0.0841881
[2725]	valid_0's binary_logloss: 0.0841881
[2726]	valid_0's binary_logloss: 0.0841914
[2727]	valid_0's binary_logloss: 0.0841912
[2728]	valid_0's binary_logloss: 0.0841927
[2729]	valid_0's binary_logloss: 0.0841952
[2730]	valid_0's binary_logloss: 0.0841887
[2731]	valid_0's binary_logloss: 0.0841906
[2732]	valid_0's binary_logloss: 0.0841905
[2733]	valid_0's binary_logloss: 0.0841904
[2734]	valid_0's binary_logloss: 0.0841963
[2735]	valid_0's binary_logloss: 0.0841935
[2736]	valid_0's binary_logloss: 0.0841896
[2737]	valid_0's binary_logloss: 0.0841874
[2738]	valid_

[2907]	valid_0's binary_logloss: 0.0841421
[2908]	valid_0's binary_logloss: 0.0841416
[2909]	valid_0's binary_logloss: 0.0841359
[2910]	valid_0's binary_logloss: 0.0841393
[2911]	valid_0's binary_logloss: 0.0841382
[2912]	valid_0's binary_logloss: 0.0841441
[2913]	valid_0's binary_logloss: 0.0841426
[2914]	valid_0's binary_logloss: 0.0841448
[2915]	valid_0's binary_logloss: 0.0841507
[2916]	valid_0's binary_logloss: 0.0841521
[2917]	valid_0's binary_logloss: 0.0841513
[2918]	valid_0's binary_logloss: 0.08415
[2919]	valid_0's binary_logloss: 0.084149
[2920]	valid_0's binary_logloss: 0.0841518
[2921]	valid_0's binary_logloss: 0.0841531
[2922]	valid_0's binary_logloss: 0.0841496
[2923]	valid_0's binary_logloss: 0.0841532
[2924]	valid_0's binary_logloss: 0.0841597
[2925]	valid_0's binary_logloss: 0.0841593
[2926]	valid_0's binary_logloss: 0.0841601
[2927]	valid_0's binary_logloss: 0.0841646
[2928]	valid_0's binary_logloss: 0.084156
[2929]	valid_0's binary_logloss: 0.0841529
[2930]	valid_0'

[3097]	valid_0's binary_logloss: 0.0840539
[3098]	valid_0's binary_logloss: 0.0840588
[3099]	valid_0's binary_logloss: 0.0840575
[3100]	valid_0's binary_logloss: 0.0840582
[3101]	valid_0's binary_logloss: 0.0840606
[3102]	valid_0's binary_logloss: 0.0840594
[3103]	valid_0's binary_logloss: 0.0840589
[3104]	valid_0's binary_logloss: 0.0840632
[3105]	valid_0's binary_logloss: 0.0840602
[3106]	valid_0's binary_logloss: 0.0840567
[3107]	valid_0's binary_logloss: 0.0840564
[3108]	valid_0's binary_logloss: 0.0840599
[3109]	valid_0's binary_logloss: 0.0840652
[3110]	valid_0's binary_logloss: 0.0840665
[3111]	valid_0's binary_logloss: 0.0840688
[3112]	valid_0's binary_logloss: 0.0840716
[3113]	valid_0's binary_logloss: 0.0840778
[3114]	valid_0's binary_logloss: 0.0840792
[3115]	valid_0's binary_logloss: 0.0840791
[3116]	valid_0's binary_logloss: 0.0840816
[3117]	valid_0's binary_logloss: 0.0840791
[3118]	valid_0's binary_logloss: 0.0840793
[3119]	valid_0's binary_logloss: 0.0840776
[3120]	vali

误差  0.0840421533701
KS  0.279803665922
